In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv' , sep = ',').drop(columns = ['Name', 'Ticket', 'Cabin','PassengerId'], axis = 1)
test_data  =  pd.read_csv('/kaggle/input/titanic/test.csv' , sep = ',').drop(columns = ['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1)
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [3]:
ohe_train = pd.get_dummies(train_data, columns = ['Sex', 'Embarked'])
ohe_test = pd.get_dummies(test_data, columns = ['Sex', 'Embarked'])

from sklearn.impute import KNNImputer
myknn = KNNImputer(n_neighbors = 7)
Y_train = train_data.Survived
train_data = pd.DataFrame(myknn.fit_transform(ohe_train.drop(axis = 1, columns = ['Survived'])))
test_data  = pd.DataFrame(myknn.transform(ohe_test))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_data,Y_train,train_size = 0.8, test_size = 0.2, random_state = 0)

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model.fit(x_train,y_train,batch_size = 50, epochs = 500, 
                    validation_data = (x_test,y_test))

In [7]:
model.evaluate(x_test,y_test)

6/6 [==============================] - 0s 4ms/step - loss: 0.5525 - accuracy: 0.7933


[0.552470862865448, 0.7932960987091064]

In [ ]:
a = model.predict(test_data, batch_size = 50)
a = np.squeeze(a, axis = 1)
a

In [9]:
for i in range(len(a)):
    if a[i] < 0.5 :
        a[i] = 0
    elif a[i] > 0.5:
        a[i] = 1
        
a = a.astype('int32')

In [10]:
result = pd.DataFrame(a,columns = ['Survived'])
passengerid = np.arange(892,1310,1, dtype = 'int32')
result['PassengerId'] = pd.Series(passengerid)
result = result[['PassengerId', 'Survived']]
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [11]:
result.to_csv('gender_submission.csv' , sep = ',', index = False)